In [1]:
#import stuff here
import math
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#The file "Stand_Data.csv" generated from "Standardization.ipynb" is required
file_dir = "Stand_Data.csv"
data = np.loadtxt(file_dir, delimiter=",")
print(np.shape(data))

(39644, 58)


# PART A:
Start with a basic linear model (the features are only standardized and are used as they are
without any transformations) regardless if you are doing the least-squares data fitting (Chapter
13) or the least-squares classification (Chapter 14) task. Evaluate the initial model using cross-
validation and report the RMS error. Make sure to save the model parameters for each fold of
the cross-validation.

In [3]:
#The file 'OnlineNewsPopularity.csv' from https://archive.ics.uci.edu/dataset/332/online+news+popularity is required
file_dir2 = 'OnlineNewsPopularity.csv'
columns = [60]
target = np.loadtxt(file_dir2, delimiter = ',', skiprows=1, usecols = columns)
print(np.shape(target))

(39644,)


##Linear Model

In [4]:
#Add a column of 1's to the right side of A
A = data
A = np.vstack((A.T, np.ones(39644))).T
print(np.shape(A))

#solve least squares problem
params = np.linalg.lstsq(A, target, rcond=None)[0]

(39644, 59)


##Cross Validation

###Calculate RMS

In [5]:
def RME(x_test, y_test, params):
  preds = []
  data_size = x_test.shape[0]
  for i in range(data_size):
    pred = np.dot(x_test[i], params[:58]) + params[58]
    preds.append(pred)
  return math.sqrt((np.sum(np.square(np.subtract(y_test, preds))))/data_size)

#print(RME(data,target,params))

###Cross Validation Folds

In [6]:
#Train on 5 folds, indexes: [0-7928, 7929-15857, 15858-23786, 23787-31715, 31716-39643]
#fold 1
x_train1 = A[7929:,:]
x_test1 = data[:7979,:]
y_train1 = target[7929:]
y_test1 = target[:7979]
params1 = np.linalg.lstsq(x_train1, y_train1, rcond=None)[0]
print(f"Fold 1 RMS Error: {RME(x_test1, y_test1, params1)}")

#fold 2
x_train2 = np.vstack((A[:7929,:], A[15858:,:]))
x_test2 = data[7929:15858,:]
y_train2 = np.concatenate((target[:7929], target[15858:]))
y_test2 = target[7929:15858]
params2 = np.linalg.lstsq(x_train2, y_train2, rcond=None)[0]
print(f"Fold 2 RMS Error: {RME(x_test2, y_test2, params2)}")

#fold 3
x_train3 = np.vstack((A[:15858,:], A[23787:,:]))
x_test3 = data[15858:23787,:]
y_train3 = np.concatenate((target[:15858], target[23787:]))
y_test3 = target[15858:23787]
params3 = np.linalg.lstsq(x_train3, y_train3, rcond=None)[0]
print(f"Fold 3 RMS Error: {RME(x_test3, y_test3, params3)}")

#fold 4
x_train4 = np.vstack((A[:23787,:], A[31716:,:]))
x_test4 = data[23787:31716,:]
y_train4 = np.concatenate((target[:23787], target[31716:]))
y_test4 = target[23787:31716]
params4 = np.linalg.lstsq(x_train4, y_train4, rcond=None)[0]
print(f"Fold 4 RMS Error: {RME(x_test4, y_test4, params4)}")

#fold 5
x_train5 = A[:31716,:]
x_test5 = data[31716:,:]
y_train5 = target[:31716]
y_test5 = target[31716:]
params5 = np.linalg.lstsq(x_train5, y_train5, rcond=None)[0]
print(f"Fold 5 RMS Error: {RME(x_test5, y_test5, params5)}")

param_matrix = np.vstack((params1, params2, params3, params4, params5))

means = []
stds = []
cvds = []


for i in range(59):
    means.append(np.mean(param_matrix[:,i]))
    stds.append(np.std(param_matrix[:,i]))
    cvds.append(round(stds[i]/abs(means[i])*100,1))

print(means)
print(stds)
print(cvds)



#take the average parameter values of the five folds
params_mean = np.zeros(59)
for i in range(59):
  params_mean[i] = (params1[i] + params2[i] + params3[i] + params4[i] + params5[i])/5
print(f"Cross Validated RMS Error: {RME(data, target, params_mean)}")
print(f"Normal Linear Regression RMS Error: {RME(data,target,params)}")

Fold 1 RMS Error: 12953.320171324256
Fold 2 RMS Error: 12239.76141359509
Fold 3 RMS Error: 15904.998478589163
Fold 4 RMS Error: 6448.062377989548
Fold 5 RMS Error: 8065.5797471211845
[193.183270692095, 279.46747001689545, 13959.435843443056, -7759.0609000437935, -5370.109808115698, 299.1010427952939, -224.46585708149786, 98.88067326565411, 22.885019535906906, -490.11339437851257, 94.2644784913228, -234.17656045742515, -446.3044874617068, -292.43269882613635, -141.16730036525624, -210.6944106415267, -196.44123246291875, 514.3853440556793, 321.578382871984, -208.15374830673235, -121.17620844733683, -36.01696960249544, -90.17674564255758, -413.8998173392596, -1236.4385680398282, 2187.165403637543, 547.1264621723764, 260.35798975654467, -174.66557257141363, 138.83817795806402, -65.58085638510757, -0.39525299816364734, -69.15928101063886, -48.523565409205865, 83.75969612422155, -10.296195703344505, 52.088638220429836, 44666.23817942976, 37128.53192643997, 47570.2329708241, 50006.87339696756

#PART B:
Perform feature engineering. Come up with more interesting feature mappings or basis functions
for the linear least-squares data fitting or the linear least-squares classifier. For example, try out
a stratified model (Chapter 13.3.2) using the results of the k-means clustering you did earlier.
Choose the best model using cross-validation and report the RMS error. Make sure to save the
model parameters for each fold of the cross-validation. If you are doing classification, report the
confusion matrix for the best model you found.

##Feature Engineering - Product Interactions

###Determining 5 least significant factors

In [7]:
#Sort the parameters by magnitude and retrieve the five features that contribute the least
params_abs = abs(np.array(params[:58])) #take magnitude of feature parameters
#attach the correlating index to teh parameter value
indexes = list(range(0,58))
stack = np.vstack((indexes, params_abs))
#sort by parameter magnitude
stack = stack.T
sorted = stack[stack[:,1].argsort()]
print(sorted[:5])
worst_indexes = sorted[:5, 0] #list of indexes of least significant features
worst_indexes = worst_indexes.astype(np.int64)

[[31.          0.435544  ]
 [35.         10.36733501]
 [53.         17.43081268]
 [45.         22.70689072]
 [ 8.         23.18702348]]


###Performing Feature Engineering
We will be taking the 5 least significant features and multipling them with all other features to create a new feature to see if it is only important when considering another feature.

In [8]:
#For each of the 5 least significant features, create a new feature by multiplying it with the 57 other features (creates a total of 285 new features)
#B represents a modified A matrix
B=data.T
print(np.shape(B))
i = 0 #i keeps track of which bad index we are on
j = 0 #j represents all other features
for i in range(5):
  bad_index = worst_indexes[i]
  for j in range(58):
    feature_added = []
    if i==j:
      continue #irrelevent to consider a non significant feature with itself
    else:
      for k in range(39644): #k represents all the data points
        feature_added.append(B[bad_index,k]*B[j,k])
      B = np.vstack((B,feature_added))
#newdata represents a modified data matrix
newdata = B.T
B = np.vstack((B,np.ones(39644)))
B = B.T
"""
print(np.shape(newdata))
print(np.shape(B))
print("done")
"""

(58, 39644)


'\nprint(np.shape(newdata))\nprint(np.shape(B))\nprint("done")\n'

##Evaluate New Features

In [9]:
def newRME(x_test, y_test, params):
  preds = []
  data_size = x_test.shape[0]
  for i in range(data_size):
    pred = np.dot(x_test[i], params[:343]) + params[343]
    preds.append(pred)
  return math.sqrt((np.sum(np.square(np.subtract(y_test, preds))))/data_size)

In [10]:
newparams = np.linalg.lstsq(B, target, rcond=None)[0]


#5 folds, indexes: [0-7928, 7929-15857, 15858-23786, 23787-31715, 31716-39643]
#fold 1
newx_train1 = B[7929:,:]
newx_test1 = newdata[:7979,:]
newy_train1 = target[7929:]
newy_test1 = target[:7979]
newparams1 = np.linalg.lstsq(newx_train1, newy_train1, rcond=None)[0]
print(f"New Fold 1 RMS Error: {newRME(newx_test1, newy_test1, newparams1)}")


#fold 2
newx_train2 = np.vstack((B[:7929,:], B[15858:,:]))
newx_test2 = newdata[7929:15858,:]
newy_train2 = np.concatenate((target[:7929], target[15858:]))
newy_test2 = target[7929:15858]
newparams2 = np.linalg.lstsq(newx_train2, newy_train2, rcond=None)[0]
print(f"New Fold 2 RMS Error: {newRME(newx_test2, newy_test2, newparams2)}")


#fold 3
newx_train3 = np.vstack((B[:15858,:], B[23787:,:]))
newx_test3 = newdata[15858:23787,:]
newy_train3 = np.concatenate((target[:15858], target[23787:]))
newy_test3 = target[15858:23787]
newparams3 = np.linalg.lstsq(newx_train3, newy_train3, rcond=None)[0]
print(f"New Fold 3 RMS Error: {newRME(newx_test3, newy_test3, newparams3)}")


#fold 4
newx_train4 = np.vstack((B[:23787,:], B[31716:,:]))
newx_test4 = newdata[23787:31716,:]
newy_train4 = np.concatenate((target[:23787], target[31716:]))
newy_test4 = target[23787:31716]
newparams4 = np.linalg.lstsq(newx_train4, newy_train4, rcond=None)[0]
print(f"New Fold 4 RMS Error: {newRME(newx_test4, newy_test4, newparams4)}")


#fold 5
newx_train5 = B[:31716,:]
newx_test5 = newdata[31716:,:]
newy_train5 = target[:31716]
newy_test5 = target[31716:]
newparams5 = np.linalg.lstsq(newx_train5, newy_train5, rcond=None)[0]
print(f"New Fold 5 RMS Error: {newRME(newx_test5, newy_test5, newparams5)}")

param_matrix_new = np.vstack((newparams1, newparams2, newparams3, newparams4, newparams5))

means_new = []
stds_new = []
cvds_new = []


for i in range(344):
    means_new.append(np.mean(param_matrix_new[:,i]))
    stds_new.append(np.std(param_matrix_new[:,i]))
    cvds_new.append(round(stds_new[i]/abs(means_new[i])*100,1))

print(means_new)
print(stds_new)
print(cvds_new)






#take the average parameter value of the five folds
newparams_mean = np.zeros(344)
for i in range(344):
 newparams_mean[i] = (newparams1[i] + newparams2[i] + newparams3[i] + newparams4[i] + newparams5[i])/5
print(f"New Cross Validated RMS Error: {newRME(newdata, target, newparams_mean)}")
print(f"New Linear Regression RMS Error: {newRME(newdata,target,newparams)}")

New Fold 1 RMS Error: 15793.41339679203
New Fold 2 RMS Error: 12375.805800802822
New Fold 3 RMS Error: 15960.51636812314
New Fold 4 RMS Error: 108183.84277710892
New Fold 5 RMS Error: 8171.10214671604
[195.25891946285066, 93.35759393984672, 2925.4989241759113, -261191776406.76114, -1305.843339504351, 317.0931353757759, -216.70028020771088, 143.96606841570366, 815.5214818361216, -207.7305447200925, 66.74221641649052, -208.77426378411275, -463.95828895084543, -231.1746720786096, -182.99082298650197, -234.23636104146908, -229.60821677701023, 536.6538551606832, 282.6261632492907, -162.3717573903346, -120.22398687440587, 24.016595369191098, -146.3543991124331, -392.36668620472153, -1142.4704176626278, 2074.7850613805035, 553.0394678724872, 152.65105249801067, -55.040680670469065, 143496746.5776508, 150794631.76618257, -558749238.1849291, 149580400.40541816, 133207316.18974653, 124378808.35247178, -113726259.48428872, 3362714.610053265, 988050.0009109497, 825290.3288318634, 1059692.136519336

In [11]:
#regularizing linear model
test_set = A[:19822]
training_set = A[19822:]

test_outcome = target[:19822]
training_outcome = target[19822:]


reg_param_values = [.001,1,10,20,100,200,300,400,600,1000,1200,1300,1500,1800,2000,2300,2600,3000,4000,5000,6000,7000,8000,9000,10000]
fit_RMS = []
param_norms = []

param_matrix_reg1 = 0
first_iter = True

for reg_param in reg_param_values:
    dim = A.shape[1]
    ident = np.identity(dim)
    ident *= reg_param
    ident[dim-1][dim-1] = 0
    mat = np.vstack((training_set, ident))

    

    outcome = np.concatenate((training_outcome, np.zeros(dim)))
    params = np.linalg.lstsq(mat, outcome, rcond=None)[0]
    param_norm = np.linalg.norm(np.delete(params,dim-1)) 
    param_norms = np.append(param_norms, param_norm)

    if first_iter:
        param_matrix_reg1 = params
        first_iter = False
    else:
        param_matrix_reg1 = np.vstack((param_matrix_reg1,params))

    preds = []
    for i in range(test_set.shape[0]):
        pred = np.dot(test_set[i],params) 
        preds.append(pred)

    dif = preds - test_outcome
    norm_ = np.linalg.norm(dif)
    fit_RMS.append(norm_/np.sqrt(dif.shape[0]))

  

    
lowest_RMS = np.argmin(fit_RMS)

print(reg_param_values[lowest_RMS])
print(fit_RMS[lowest_RMS])
print(param_norms[lowest_RMS])


means_reg1 = []
stds_reg1 = []
cvds_reg1 = []


for i in range(param_matrix_reg1.shape[1]):
    means_reg1.append(np.mean(param_matrix_reg1[:,i]))
    stds_reg1.append(np.std(param_matrix_reg1[:,i]))
    cvds_reg1.append(round(stds_reg1[i]/abs(means_reg1[i])*100,1))

print(means_reg1)
print(stds_reg1)
print(cvds_reg1)




20
13821.7649707273
2386.922700781445
[52.00683195748823, 39.43568464058317, 1569.888626639391, -195.54026199791136, -1047.5923342836215, 60.11618853541886, -33.01402936459122, 26.91718131378941, -18.041208522885913, -106.17574869181159, 26.973865932429497, -57.90519063939142, -87.16181286684962, -65.16444260088917, -15.29073421156404, -15.504997570403765, -27.889521097224087, -18.410533565615687, 68.26734110993068, -82.37216093932902, -12.102220423021912, 36.92161214352756, -27.492309806658255, -48.969232436413506, -190.24247776457102, 401.59599562572095, -1.6483644151079528, 3.383522296994582, 55.26680882266205, 32.013288543561714, -4.076198504930643, -3.2456216305993593, -24.605219452582563, -13.296723676039955, 4.97277243132869, 15.276479198854165, 15.03377266909779, 16823.273755832124, 14028.218500941748, 17958.90286857779, 18885.09538548753, 18480.227626980923, 35.79736880194075, -30.752736401798856, -32.28347471168885, 5.617908042625137, 72.41340323551414, 35.95990479659461, -0.

In [12]:
#regularizing linear model
test_set = B[:19822]
training_set = B[19822:]

test_outcome = target[:19822]
training_outcome = target[19822:]


reg_param_values = [.001,1,10,20,100,200,300,400,600,1000,1200,1300,1500,1800,2000,2300,2600,3000,4000,5000,6000,7000,8000,9000,10000]
fit_RMS = []
param_norm = []

param_matrix_reg2 = 0
first_iter = True

for reg_param in reg_param_values:
    dim = B.shape[1]
    ident = np.identity(dim)
    ident *= reg_param
    ident[dim-1][dim-1] = 0
    mat = np.vstack((training_set, ident))

    

    outcome = np.concatenate((training_outcome, np.zeros(dim)))
    params = np.linalg.lstsq(mat, outcome, rcond=None)[0]
    param_norm = np.linalg.norm(np.delete(params,dim-1)) 
    param_norms = np.append(param_norms, param_norm)

    if first_iter:
        param_matrix_reg2 = params
        first_iter = False
    else:
        param_matrix_reg2 = np.vstack((param_matrix_reg2,params))
    preds = []
    for i in range(test_set.shape[0]):
        pred = np.dot(test_set[i],params) 
        preds.append(pred)

    dif = preds - test_outcome
    norm_ = np.linalg.norm(dif)
    fit_RMS.append(norm_/np.sqrt(dif.shape[0]))
    
lowest_RMS = np.argmin(fit_RMS)

print(reg_param_values[lowest_RMS])
print(fit_RMS[lowest_RMS])
print(param_norms[lowest_RMS])


means_reg2 = []
stds_reg2 = []
cvds_reg2 = []


for i in range(param_matrix_reg2.shape[1]):
    means_reg2.append(np.mean(param_matrix_reg2[:,i]))
    stds_reg2.append(np.std(param_matrix_reg2[:,i]))
    cvds_reg2.append(round(stds_reg2[i]/abs(means_reg2[i])*100,1))

print(means_reg2)
print(stds_reg2)
print(cvds_reg2)



100
13834.784062169654
847.583449965849
[54.50748341468537, 26.319917306369597, 1643.4246921634071, 73.49183221614187, -1145.1859762097615, 51.92659409928596, -23.60947893221815, 30.252216719707512, -32.68447345520208, -98.58469143707659, 23.618946708474933, -61.2293173610137, -87.58252105199652, -69.22648627010803, -17.2949922909852, -20.019658263711804, -24.3102015398288, 129.25623801351583, 65.805937474899, -78.39358891554085, -12.204352260928927, 70.33627447275231, -29.56299798017645, -49.87402304115105, -178.82508171163514, 391.1849744999851, 23.196792475503266, 30.388120402263088, 17.717954102605162, 26.31878747555205, -2.003136841430219, -0.439434458349596, -18.972885581040895, -10.689035727989523, 7.254679402409964, 1.7956064461629508, 6.5313973649545565, -1689.2292640247429, -1436.5357387263273, -1899.9634423410646, -1899.6465178605247, -1865.2461290718834, 25.069657129449247, -33.44776049947645, -30.86006472339485, -59.874029129466564, -8.53262777078447, -36.326125952976014, 

In [13]:
#New Feature Engineering

new_matrix = A

print(A.shape)

for i in range(A.shape[0]):
    if A[i][7] > 0:
        A[i][7] = 1
    else:
        A[i][7] = 0

    if A[i][8] > 0:
        A[i][8] = 1
    else:
        A[i][8] = 0

    if A[i][26] > 0:
        A[i][26] = np.log(A[i][26])

    if A[i][27] > 0:
        A[i][27] = np.log(A[i][27])

    if A[i][28] > 0:
        A[i][28] = np.log(A[i][28])
    

#regularizing linear model
test_set = new_matrix[:19822]
training_set = new_matrix[19822:]

test_outcome = target[:19822]
training_outcome = target[19822:]


reg_param_values = [.001,1,10,20,100,200,300,400,600,1000,1200,1300,1500,1800,2000,2300,2600,3000,4000,5000,6000,7000,8000,9000,10000]
fit_RMS = []
param_norms = []


first_iter = True
param_matrix_reg3 = 0


for reg_param in reg_param_values:
    dim = A.shape[1]
    ident = np.identity(dim)
    ident *= reg_param
    ident[dim-1][dim-1] = 0
    mat = np.vstack((training_set, ident))

    

    outcome = np.concatenate((training_outcome, np.zeros(dim)))
    params = np.linalg.lstsq(mat, outcome, rcond=None)[0]
    param_norm = np.linalg.norm(np.delete(params,dim-1)) 
    param_norms = np.append(param_norms, param_norm)


    preds = []
    for i in range(test_set.shape[0]):
        pred = np.dot(test_set[i],params) 
        preds.append(pred)

    dif = preds - test_outcome
    norm_ = np.linalg.norm(dif)
    fit_RMS.append(norm_/np.sqrt(dif.shape[0]))

    if first_iter:
        param_matrix_reg3 = params
        first_iter = False
    else:
        param_matrix_reg3 = np.vstack((param_matrix_reg3,params))


    
lowest_RMS = np.argmin(fit_RMS)

print(reg_param_values[lowest_RMS])
print(fit_RMS[lowest_RMS])
print(param_norms[lowest_RMS])



means_reg3 = []
stds_reg3 = []
cvds_reg3 = []


for i in range(param_matrix_reg3.shape[1]):
    means_reg3.append(np.mean(param_matrix_reg3[:,i]))
    stds_reg3.append(np.std(param_matrix_reg3[:,i]))
    cvds_reg3.append(round(stds_reg3[i]/abs(means_reg3[i])*100,1))

print(means_reg3)
print(stds_reg3)
print(cvds_reg3)




(39644, 59)
20
13828.735274427861
2379.171643289668
[51.76924281159137, 37.50891976552302, 1398.4389934069186, -226.37735604578236, -930.0982463298245, 57.62207402886672, -37.20146064572228, 57.55156183346348, 44.22835944258739, -96.28382885900186, 25.834301353555265, -58.35704954332379, -89.96669426716144, -64.33838180718615, -16.64257339851361, -12.492460688394814, -29.909484633676875, -18.377179230204785, 62.70499926872657, -78.06871666941161, -11.181131538072542, 38.30837422020492, -27.686629026211193, -48.734173267296846, -183.3222597293334, 399.5288562715434, -19.364734136497578, -16.771005571329525, -39.06787417483883, 32.37901938663536, -4.657383419497517, -3.312440678344375, -24.752592754156026, -12.759972221765803, 4.2634554574545, 15.888780175651275, 14.987366133922432, 12401.726261544616, 10334.319661306768, 13214.701837689192, 13912.616536122392, 13617.841710830375, 38.07091540412984, -30.616240525757476, -35.10242943826686, 1.717279220512515, 64.33965319885628, 30.3098153